# **AI Candidate Screening Agent using CrewAI and Gemini 2.5 Flash**

This agent help you analyze your CV against the job requirements, provide result for the next recruitment step, create rejection/interview email and create interview question.

In [1]:
!pip install crewai google-genai pypdf boto3 pydantic

In [2]:
# Retrieve AWS credentials from Colab Secrets
from google.colab import userdata
import os
os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWSACCESSKEY')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWSSECRETKEY')

In [3]:
# Retrieve Gemini API Key from AWS Secret Manager
import boto3
import json
secretmanager = boto3.client('secretsmanager', region_name="us-west-2")
response = secretmanager.get_secret_value(SecretId='geminiapikey')
secret_json = json.loads(response["SecretString"])
api_key = secret_json["GEMINI_API_KEY"]

In [4]:
from crewai import LLM, Agent, Task, Crew, Process
from crewai.tools import tool
from pydantic import BaseModel, Field
from typing import Optional
import datetime
import json
import random
llm = LLM(model="gemini/gemini-2.5-flash", api_key=api_key, temperature=0.1)

# **Structure of this AI Agent**

**User --> Extract CV --> Compare and Match between Job Requirements and CV --> Result to the next step of the recruitment process --> Create Interview Email --> Create Interview Question --> End OR Create Rejection Email -> End**

# **Agent 1: Compare and Match between Job Requirements and CV**

In [5]:
class compareMatchClass(BaseModel):
    """Compare and match between job requirements and curriculum vitae."""
    minimal_requirements_analysis: str = Field(description="Analysis of minimal requirements")
    preferred_requirements_analysis: str = Field(description="Analysis of preferred requirements")
    strengths: str = Field(description="Strengths of candidate")
    potential_gaps: str = Field(description="Potential gaps of candidate")
    candidate_name: str = Field(description="Candidate name in the CV")

compareAndMatchAgent = Agent(
    role="Compare and Match Agent",
    goal="Compare and match between job requirements and extracted of curriculum vitae",
    backstory="""
    You are an expert HR recruiter with a keen eye for detail. Your primary function is to compare and match a candidate curriculum vitae (CV) against job requirements.
    Your insights are crucial for determining a candidate's suitability for the next stages of the recruitment process.
    """,
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm,
    output_pydantic=compareMatchClass
)

compareAndMatchTask = Task(
    description="""
    You are a virtual human resources expert. You help me COMPARE and MATCH between {cv_data} and AI engineer position job requirements like this :
    1. Work across the AI lifecycle: from data preparation and model development to evaluation and deployment.
    2. Fine-tune and integrate LLMs (like OpenAI and Gemini) into ERP workflows.
    3. Build smart features such as recommendation engines, forecasting modules, NLP tools, and more.
    Preferred requirements :
    Develop and maintain scalable cloud-based AI solutions across multi-cloud platforms (AWS, GCP, Azure).
    """,
    expected_output="""
    Minimal Requirements Analysis :

    Requirement 1:

    Requirement n (n is the number of requirement):

    Preferred Requirement Analysis :

    Strengths:

    Potential Gaps:

    Candidate Name:
    """,
    agent=compareAndMatchAgent
)

# **Agent 2: Score to the Next Step of the Recruitment Process**



In [6]:
class scoreNextStepClass(BaseModel):
    """Score value to proceed to the next step in the recruitment process."""
    score: int = Field(description="Score of candidate matched")

scoreNextStepAgent = Agent(
    role="Score to Next Step of the Recruitment Process Agent",
    goal="Provide score to next step of the recruitment process",
    backstory="""
    You are an expert in HR recruitment processes, capable of assessing candidate suitability and determining the next logical step in their application journey.
    """,
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm,
    output_pydantic=scoreNextStepClass
)

scoreNextStepTask = Task(
    description="""
    You are a virtual human resources expert. You help me add value from 0 until 10 to the next step of recruitment process.
    Answer with a number between 0 and 10 ONLY without any additions.
    """,
    expected_output="""
    1
    """,
    agent=scoreNextStepAgent
)

# **Agent 3: Create Rejection/Interview Email**

In [7]:
class createEmailClass(BaseModel):
    """Create rejection email or interview invitation email for candidate."""
    email: str = Field(description="Body of email for candidate")

createEmailAgent = Agent(
    role="Create Rejection or Interview Email Agent",
    goal="Crate rejection or interview email to next step of the recruitment process",
    backstory="""
    You are an expert in HR recruitment processes, capable of writing rejection or interview email for candidate.
    """,
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm,
    output_pydantic=createEmailClass
)

createEmailTask = Task(
    description="""
    You are a virtual human resources expert. You are help me create email in the recruitment process.

    If the score is LESS THAN 7 out of 10, create a very simple rejection about failed to AI engineer position email for unsuccessful candidate.
    Write output structure like this:
    Hello, {compareAndMatchTask.output.candidate_name}\n
    ........\n
    Thanks,\n
    HRD of AgentCore.

    Otherwise, create an interview email for a candidate accepted to the next step to AI engineer position. Write output structure like this:
    Hello, {compareAndMatchTask.output.candidate_name}\n
    ........\n
    Date : {interview_date}\n
    Time : {interview_time}\n
    Google Meet interview link : https://bit.ly/agentcore-interview\n
    ........\n
    Thanks,\n
    HRD of AgentCore.
    """,
    expected_output="""
    Hello, {compareAndMatchTask.output.candidate_name}\n
    ........\n
    Thanks,\n
    HRD of AgentCore.

    OR

    Hello, {compareAndMatchTask.output.candidate_name}\n
    ........\n
    Date : {interview_date}\n
    Time : {interview_time}\n
    Google Meet interview link : https://bit.ly/agentcore-interview\n
    ........\n
    Thanks,\n
    HRD of AgentCore.
    """,
    agent=createEmailAgent,
    context=[compareAndMatchTask, scoreNextStepTask]
)

# **Agent 4: Create Interview Questions**

In [8]:
class createInterviewQuestionClass(BaseModel):
    """Create interview questions for candidates who are accepted to the next step."""
    questions: str = Field(description="Interview questions for candidate")

createInterviewQuestionAgent = Agent(
    role="Create Interview Question Agent",
    goal="Create interview question for interview session of the recruitment process",
    backstory="""
    You are an experienced HR interviewer, skilled at formulating insightful and relevant questions.
    Your objective is to design interview questions that effectively probe a candidate's skills, experience, and suitability based on their curriculum vitae.
    """,
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm,
    output_pydantic=createInterviewQuestionClass
)

createInterviewQuestionTask = Task(
    description="""
    You are a virtual human resources expert. You are help me create 3 interview questions about CV PDF file that already extracted.

    If the score is LESS THAN 7 out of 10, DO NOT create interview questions. Write output structure like this:
    {\"questions\": \"-\"}

    Otherwise, create 3 interview questions about CV PDF file that already extracted. Write output structure like this:
    QUESTION 1 : ........ \n
    QUESTION 2 : ........ \n
    QUESTION 3 : ........ \n
    """,
    expected_output="""
    {\"questions\": \"-\"}

    OR

    QUESTION 1 : ........ \n
    QUESTION 2 : ........ \n
    QUESTION 3 : ........ \n
    """,
    agent=createInterviewQuestionAgent,
    context=[compareAndMatchTask, scoreNextStepTask]
)

In [9]:
# Create AI agent crew
candidate_screening_crew = Crew(
    agents=[compareAndMatchAgent, scoreNextStepAgent, createEmailAgent, createInterviewQuestionAgent],
    tasks=[compareAndMatchTask, scoreNextStepTask, createEmailTask, createInterviewQuestionTask],
    process=Process.sequential,
    verbose=True,
)

In [10]:
def createEmail():
    # Formatted interview date (today + 3 days)
    today = datetime.date.today()
    interview_date = today + datetime.timedelta(days=3)
    formatted_date = interview_date.strftime("%d-%m-%Y")

    # Formatted interview time between 1 PM (13:00) and 4 PM (16:00)
    hour = random.randint(13, 16)
    minute = random.choice([0, 30])
    if hour == 16 and minute == 30:
        minute = 0 # if 4:30 PM was chosen, adjust to 4:00 PM
    interview_time = datetime.time(hour, minute, 0)
    formatted_time = interview_time.strftime("%I:%M %p") # Format to HH:MM AM/PM
    return formatted_date, formatted_time

formatted_date, formatted_time = createEmail()

# **Execute First Curriculum Vitae**

In [11]:
# Extract CV PDF file
from pypdf import PdfReader
reader = PdfReader("Always Winner CV.pdf")
page = reader.pages[0]
firsttext = page.extract_text()

In [12]:
firstresult = candidate_screening_crew.kickoff(inputs={
    'cv_data': firsttext,
    'interview_date': formatted_date,
    'interview_time': formatted_time
})
print(firstresult)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 32e23d3a-bb91-466c-8b75-0e528d825abe                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Compare and Match Agent                                                                                 │
│                                                                                                                 │
│  Task:                                                                                                          │
│      You are a virtual human resources expert. You help me COMPARE and MATCH between Name  :  Always  Winner    │
│  Phone  Number  :  000888999                                                                                    │
│  Email  Address  :  alwayswinner@gmail.com                                                                      │
│  Work  Experience  :                                                                                            │
│  AI  Engineer,  Winner,  LLC  |  New  York  City,  NY  |  January  1,  2025  –  Present                         │
│  Junior  AI  Engineer,  Nerwin,  LLC  |  New  York  City,  NY  |  January  1,  2024  –  December  31,  2024     │
│  Education  :                                                                                                   │
│  Bachelor  of  Artificial  Intelligence  |  University  of  AI  Everywhere  |  New  York  City,  NY  |  2023    │
│  Technical  Skills  :                                                                                           │
│  ●  Programming  Languages:  Python  (TensorFlow,  PyTorch,  Keras,  scikit-learn,  Hugging                     │
│  Face                                                                                                           │
│                                                                                                                 │
│  Transformers),                                                                                                 │
│                                                                                                                 │
│  Java,                                                                                                          │
│                                                                                                                 │
│  C++                                                                                                            │
│   ●  Generative  AI:  GANs  (StyleGAN,  BigGAN),  VAEs,  Diffusion  Models,  Autoregressive                     │
│  Models,                                                                                                        │
│                                                                                                                 │
│  LLMs                                                                                                           │
│                                                                                                                 │
│  (GPT,                                                                                                          │
│                                                                                                                 │
│  BERT,                                                                                                          │
│                                                                                                                 │
│  LLaMA),                                                                                                        │
│                                                                                                                 │
│  Stable                                                                                                         │
│                                                        

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Compare and Match Agent                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Minimal Requirements Analysis :                                                                                │
│                                                                                                                 │
│  Requirement 1:                                                                                                 │
│  Match: Strong match. The candidate explicitly lists "Data Engineering" which directly addresses data           │
│  preparation. Their comprehensive "Machine Learning" skills (Supervised Learning, Unsupervised Learning,        │
│  Transfer Learning, Model Optimization, Hyperparameter Tuning) and "Generative AI" expertise (GANs, VAEs,       │
│  Diffusion Models, Autoregressive Models, LLMs) demonstrate robust model development capabilities. The          │
│  inclusion of "MLOps" as a skill, along with tools like MLflow, Weights & Biases, and Optuna, confirms          │
│  experience in evaluation, deployment, and ongoing lifecycle management. Their work experience as an AI         │
│  Engineer further validates practical application across these stages.                                          │
│                                                                                                                 │
│  Requirement 2:                                                                                                 │
│  Match: Good match. The candidate possesses strong expertise in "Generative AI," specifically listing "LLMs     │
│  (GPT, BERT, LLaMA)." GPT is a prominent OpenAI model, indicating familiarity with key LLM providers. While     │
│  "Gemini" is not explicitly mentioned, the broad LLM knowledge suggests a strong foundation for working with    │
│  various models. The "API Development" skill is crucial for integrating LLMs into different systems. The        │
│  primary area not explicitly covered is direct experience with "ERP workflows" specifically, but the core       │
│  technical ability to fine-tune and integrate LLMs is clearly present.                                          │
│                                                                                                                 │
│  Requirement 3:                                                                                                 │
│  Match: Strong match. The candidate's "Machine Learning" skills, including Supervised and Unsupervised          │
│  Learning, provide the necessary foundation for building recommendation engines and forecasting modules. Their  │
│  proficiency in "Python (TensorFlow, PyTorch, Keras, scikit-learn, Hugging Face Transformers)" and "Generative  │
│  AI (LLMs: GPT, BERT, LLaMA)" directly supports the development of advanced Natural Language Processing (NLP)   │
│  tools. Their professional experience as an AI Engineer further implies practical experience in creating such   │
│  smart features.                                                                                                │
│                                                                                                                 │
│  Preferred Requirement Analysis :                                                                               │
│  Match: Excellent match. The candidate explicitly lists "Cloud Platforms: AWS (Sagemaker, EC2, S3), Azure       │
│  (Azure ML), Google Cloud Platform (Vertex AI)," direct

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Create Interview Question Agent                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│      You are a virtual human resources expert. You are help me create 3 interview questions about CV PDF file   │
│  that already extracted.                                                                                        │
│                                                                                                                 │
│      If the score is LESS THAN 7 out of 10, DO NOT create interview questions. Write output structure like      │
│  this:                                                                                                          │
│      {"questions": "-"}                                                                                         │
│                                                                                                                 │
│      Otherwise, create 3 interview questions about CV PDF file that already extracted. Write output structure   │
│  like this:                                                                                                     │
│      QUESTION 1 : ........                                                                                      │
│                                                                                                                 │
│      QUESTION 2 : ........                                                                                      │
│                                                                                                                 │
│      QUESTION 3 : ........                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Create Interview Question Agent                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  QUESTION 1 : Your CV highlights strong expertise in Generative AI and MLOps across multiple cloud platforms.   │
│  Can you walk us through a complex project where you were responsible for deploying and managing a large        │
│  language model in a production environment, detailing the MLOps strategies and tools you implemented to        │
│  ensure its performance, scalability, and maintainability?                                                      │
│                                                                                                                 │
│  QUESTION 2 : We're looking to integrate advanced AI capabilities into our existing business processes,         │
│  potentially involving ERP systems. Given your strong LLM integration skills, how would you approach designing  │
│  and implementing a solution to embed a fine-tuned LLM into an enterprise workflow, specifically addressing     │
│  challenges like data security, real-time performance, and seamless system interoperability?                    │
│                                                                                                                 │
│  QUESTION 3 : Your experience spans various LLMs and major cloud platforms. If a future project required you    │
│  to work with a specific LLM not explicitly listed on your CV, such as Google's Gemini, or deploy on a cloud    │
│  service you've had less hands-on experience with, how would you approach quickly acquiring the necessary       │
│  expertise and ensuring a successful project outcome?                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

QUESTION 1 : Your CV highlights strong expertise in Generative AI and MLOps across multiple cloud platforms. Can you walk us through a complex project where you were responsible for deploying and managing a large language model in a production environment, detailing the MLOps strategies and tools you implemented to ensure its performance, scalability, and maintainability?

QUESTION 2 : We're looking to integrate advanced AI capabilities into our existing business processes, potentially involving ERP systems. Given your strong LLM integration skills, how would you approach designing and implementing a solution to embed a fine-tuned LLM into an enterprise workflow, specifically addressing challenges like data security, real-time performance, and seamless system interoperability?

QUESTION 3 : Your experience spans various LLMs and major cloud platforms. If a future project required you to work with a specific LLM not explicitly listed on your CV, such as Google's Gemini, or deploy on a c

# **Execute Second Curriculum Vitae**

In [13]:
# Extract CV PDF file
from pypdf import PdfReader
reader = PdfReader("Sonny Wawwak CV.pdf")
page = reader.pages[0]
secondtext = page.extract_text()

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Crew: crew
├── 📋 Task: 8f60ed38-51a9-4b69-834b-476cdb6cbdb4
│   Assigned to: Compare and Match Agent
│   Status: ✅ Completed
├── 📋 Task: 7f5c3992-02ec-4d45-8f8f-8fb7c7a1ab72
│   Assigned to: Score to Next Step of the Recruitment Process Agent
│   Status: ✅ Completed
├── 📋 Task: acf7af63-3d8b-44c0-a93b-12911a8f8cd7
│   Assigned to: Create Rejection or Interview Email Agent
│   Status: ✅ Completed
└── 📋 Task: f4ffa05b-9997-4958-b936-aeab89f880a4
    Assigned to: Create Interview Question Agent
    Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f4ffa05b-9997-4958-b936-aeab89f880a4                                                                     │
│  Agent: Create Interview Question Agent                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [15]:
secondresult = candidate_screening_crew.kickoff(inputs={
    'cv_data': secondtext,
    'interview_date': formatted_date,
    'interview_time': formatted_time
})
print(secondresult)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 32e23d3a-bb91-466c-8b75-0e528d825abe                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Compare and Match Agent                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Minimal Requirements Analysis :                                                                                │
│                                                                                                                 │
│  Requirement 1: Work across the AI lifecycle: from data preparation and model development to evaluation and     │
│  deployment.                                                                                                    │
│  Analysis: Sonny Wawwak's CV demonstrates expertise in UI/UX design, including UX Research & Analysis, User     │
│  Flows, and Usability Testing, which are related to understanding user needs and evaluating design              │
│  effectiveness. However, there is no mention of skills or experience in data preparation, AI model              │
│  development, machine learning algorithms, or the technical aspects of AI model evaluation and deployment. His  │
│  background is in visual communication design and user experience, not AI engineering.                          │
│  Match: No.                                                                                                     │
│                                                                                                                 │
│  Requirement 2: Fine-tune and integrate LLMs (like OpenAI and Gemini) into ERP workflows.                       │
│  Analysis: Sonny Wawwak's CV does not contain any information or skills related to Large Language Models        │
│  (LLMs), specific platforms like OpenAI or Gemini, or experience with Enterprise Resource Planning (ERP)        │
│  systems. His technical skills are limited to basic HTML/CSS and a wide array of design tools.                  │
│  Match: No.                                                                                                     │
│                                                                                                                 │
│  Requirement 3: Build smart features such as recommendation engines, forecasting modules, NLP tools, and more.  │
│  Analysis: Sonny Wawwak's CV does not indicate any experience or skills in developing or implementing           │
│  AI-driven features such as recommendation engines, forecasting modules, or Natural Language Processing (NLP)   │
│  tools. His professional experience and skills are exclusively focused on the design and user experience        │
│  aspects of software development.                                                                               │
│  Match: No.                                                                                                     │
│                                                                                                                 │
│  Preferred Requirement Analysis :                                                                               │
│  Develop and maintain scalable cloud-based AI solutions across multi-cloud platforms (AWS, GCP, Azure).         │
│  Analysis: Sonny Wawwak's CV does not mention any experience with cloud platforms (AWS, GCP, Azure) or the      │
│  development and maintenance of scalable AI solutions. His technical background does not align with cloud       │
│  infrastructure or AI solution architecture.                                                                    │
│  Match: No.                                            

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8f60ed38-51a9-4b69-834b-476cdb6cbdb4                                                                     │
│  Agent: Compare and Match Agent                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Create Interview Question Agent                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"questions": "-"}                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

{"questions": "-"}


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 32e23d3a-bb91-466c-8b75-0e528d825abe                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {"questions": "-"}                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯